In [ ]:
import logging 
logging.basicConfig(level=logging.INFO)
log = logging.getLogger("my-logger")
log.setLevel(logging.INFO)
log.info("Hello World")
log.warning("this is a warning")

In [ ]:
# Test AIReady
import logging
logging.basicConfig(level=logging.INFO)

aiurl = 'http://aiapiplastico-dev.westeurope.cloudapp.azure.com'
isAIready = AIready(f'{aiurl}:5000')

In [ ]:
# Test getPrediction
blob_video_name = '28022020_Boudigau_4_short_480.mov'  
aiurl = 'http://aiapiplastico-dev.westeurope.cloudapp.azure.com'
prediction = getPrediction(blob_video_name,aiurl)

In [ ]:
str(prediction)

In [ ]:
# import prerequesite for blob
from azure.storage.blob import ContainerClient
from azure.storage.blob import BlobClient
from modules.blob import blob_in_container,blob_infos,download_blob
# import prerequesite for ai
import json
import requests
import logging
from modules.ai import is_ai_ready,get_prediction,json_prediction,get_trash_label,map_label_2_trash_id_PG,get_trash_time_index,get_trash_time_stamp
# import prerequesite for gps
import gpxpy
import gpxpy.gpx
import json
import subprocess
import datetime
from datetime import datetime
from datetime import timedelta
from shapely.geometry import Point
from functools import partial
import pyproj
from shapely.ops import transform
from tqdm import tqdm
from modules.gps import extract_gpx_from_gopro,parse_gpx,get_gps_point_list,create_time,create_latitude,create_longitude,create_elevation,fill_gps,long_lat_to_shape_point,transform_geo
# import prerequesite from media
from modules.media import get_media_duration,get_media_fps
# import prerequesite for postgre
import os
import psycopg2
from modules.postgre import get_pg_connection_string,open_pg_connection,close_pg_connection,insert_trash
import warnings
warnings.filterwarnings('ignore')
# import argparse to pass parameters to main function
import argparse

In [ ]:
# Test jsonPrediction
import json
with open('../data/prediction.json') as json_file:
    jsonPred = json.load(json_file)
jsonPred

In [ ]:
timeindex = get_trash_time_index(frametobox0)
mediafps = get_media_fps('/tmp/28022020_Boudigau_4.MP4')
timestamp = get_trash_time_stamp(timeindex,mediafps)
print(timestamp)

In [ ]:
video_name = '28022020_Boudigau_4.MP4'
gpx_path = extract_gpx_from_gopro(f'/tmp/{video_name}')
# GPX parsing
gpx_data = parse_gpx(gpx_path)

# GPS Points
gps_points = get_gps_point_list(gpx_data)

# Video duration
logger.info("\n")
video_duration = get_media_duration(f'/tmp/{video_name}')
logger.info(f'Video duration in second from metadata:{video_duration}')
media_fps = get_media_fps(f'/tmp/{video_name}')

# GPS file duration
timestampDelta = gps_points[len(gps_points)-1]['Time'] - gps_points[0]['Time']
logger.info(f'GPS file time coverage in second:{timestampDelta.seconds}')

logger.info('###################### Pipeline Step2 ######################')
logger.info('################## Add missing GPS points ##################')
video_duration_sup = int(video_duration)+1
gps_points_filled = fill_gps(gps_points,video_duration_sup)

In [ ]:
media_fps = get_media_fps('/tmp/28022020_Boudigau_4.MP4')
# Get connection string information from env variables
pg_conn_string = get_pg_connection_string()
# Open pgConnection
pg_connection = open_pg_connection(pg_conn_string)
# Create Cursor
pg_cursor = pg_connection.cursor()

import logging
logger = logging.getLogger()

row_id_list = []
for prediction in tqdm(jsonPred['detected_trash']):
    timestamp = get_trash_time_stamp(prediction,media_fps)
    trash_gps = gps_points_filled[timestamp]
    shape_trash_gps = long_lat_2_shape_point(trash_gps)
    geo_2154 = transform_geo(shape_trash_gps)
    geo_2154_trash_gps = {'Time': shape_trash_gps['Time'], 'the_geom': geo_2154,'Latitude':shape_trash_gps['Latitude'],'Longitude':shape_trash_gps['Longitude'], 'Elevation': shape_trash_gps['Elevation']}
    # get trash_type from ai module
    label = get_trash_label(prediction)
    trash_type = map_label_2_trash_id_PG(label)
    # insert trash from postgre module
    row_id = insert_trash(geo_2154_trash_gps,trash_type,pg_cursor,pg_connection)
    row_id_list.append(row_id)

In [ ]:
%env PGSERVER=pgdb-plastico-dev.postgres.database.azure.com
%env PGDATABASE=plastico-dev
%env PGUSERNAME=writer_user@pgdb-plastico-dev
%env PGPWD=SurfWriter!

In [ ]:
# import prerequesite for ai
import json
import requests
import logging
from utils.ai import is_ai_ready,get_prediction,get_json_prediction,get_trash_label,map_label_to_trash_id_PG,get_trash_time_index,get_trash_first_time
# import prerequesite for gps
import gpxpy
import gpxpy.gpx
import json
import subprocess
import datetime
from datetime import datetime
from datetime import timedelta
from shapely.geometry import Point
from functools import partial
import pyproj
from shapely.ops import transform
from tqdm import tqdm
from utils.gps import extract_gpx_from_gopro,parse_gpx,get_gps_point_list,create_time,create_latitude,create_longitude,create_elevation,fill_gps,long_lat_to_shape_point,transform_geo
# import prerequesite from media
from utils.media import get_media_duration,get_media_fps
# import prerequesite for postgre
import os
import psycopg2
from utils.postgre import get_pg_connection_string,open_pg_connection,close_pg_connection,insert_trash

In [ ]:
aiurl = 'http://aiapiplastico-dev.westeurope.cloudapp.azure.com'
prediction = get_prediction('28022020_Boudigau_4_short_480.mov',f'{aiurl}:5000')

In [ ]:
jsonPrediction = get_json_prediction(prediction)

In [ ]:
with open('../data/prediction.json') as json_file:
    jsonPrediction = json.load(json_file)

In [ ]:
video_name = '28022020_Boudigau_4.MP4'
prediction = jsonPrediction['detected_trash'][0]
media_fps = get_media_fps(f'/tmp/{video_name}')
gpx_path = extract_gpx_from_gopro(f'/tmp/{video_name}')
gpx_data = parse_gpx(gpx_path)
gps_points = get_gps_point_list(gpx_data)
video_duration = get_media_duration(f'/tmp/{video_name}')
video_duration_sup = int(video_duration)+1
gps_points_filled = fill_gps(gps_points,video_duration_sup)
time_index = get_trash_time_index(prediction,media_fps)
# trash_gps
trash_gps = gps_points_filled[time_index]
shape_trash_gps = long_lat_to_shape_point(trash_gps)
geo_2154 = transform_geo(shape_trash_gps)
geo_2154_trash_gps = {'Time': shape_trash_gps['Time'], 'the_geom': geo_2154,'Latitude':shape_trash_gps['Latitude'],'Longitude':shape_trash_gps['Longitude'], 'Elevation': shape_trash_gps['Elevation']}
# get trash_type from ai module
label = get_trash_label(prediction)
trash_type = map_label_to_trash_id_PG(label)

In [ ]:
trash_gps

In [ ]:
trash_type

In [ ]:
jsonPrediction['detected_trash']

In [ ]:
prediction = jsonPrediction['detected_trash'][0]
print(prediction)
index = str(get_trash_time_index(prediction))
clean_frame_to_box = {index:prediction['frame_to_box'][index]}
clean_prediction = {'frame_to_box':clean_frame_to_box,'id':prediction['id'],'label':prediction['label']}
print(clean_prediction)

In [ ]:
def get_clean_prediction(prediction:dict)->dict:
    index = str(get_trash_time_index(prediction))
    clean_frame_to_box = {index:prediction['frame_to_box'][index]}
    clean_prediction = {'frame_to_box':clean_frame_to_box,'id':prediction['id'],'label':prediction['label']}
    return clean_prediction

In [ ]:
prediction = jsonPrediction['detected_trash'][0]
index = str(get_trash_time_index(prediction))
clean_frame_to_box = {index:prediction['frame_to_box'][index]}
clean_prediction = {'frame_to_box':clean_frame_to_box,'id':prediction['id'],'label':prediction['label']}

In [ ]:
prediction['frame_to_box'][index]

In [ ]:
prediction = jsonPrediction['detected_trash'][0]
clean_prediction = get_clean_prediction(prediction)

In [ ]:
print(prediction)
print(clean_prediction)

In [ ]:
clean_prediction_list = []
for prediction in jsonPrediction['detected_trash']:
    clean_prediction = get_clean_prediction(prediction)
    clean_prediction_list.append(clean_prediction)

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd
pd_prediction = pd.DataFrame(clean_prediction_list)

In [ ]:
pd_prediction   

In [ ]:
prediction = jsonPrediction['detected_trash'][0]
clean_prediction = get_clean_prediction(prediction)
clean_prediction

In [ ]:
def get_trash_first_time(trash:dict)->int:
    """Get the time index for a trash, the first time it is identified

    Arguments:
        trash {dict} -- [description]

    Returns:
        int -- the index when the trash is identified for the first time
    """
    frame_to_box = trash['frame_to_box']    
    return int(list(frame_to_box.keys())[0])

def get_trash_time_index(prediction:dict,media_fps:float)->int:
    """ Get trash time stamp

    Arguments:
        prediction {dict} -- the prediction made by AI of a unique trash
        media_fps {float} -- the FPS of the media where the trash comes from

    Returns:
        timestamp -- the timestamp of the trash with regard to video it comes from
    """
    time_index = get_trash_first_time(prediction)
    timestamp = int(time_index / media_fps)
    return timestamp



In [ ]:
def get_clean_timed_prediction(prediction:dict)->dict:
    """Get clean prediction with single frame_to_box

    Arguments:
        prediction {dict} -- a single prediction from a dictionary of AI predictions

    Returns:
        clean_prediction -- a prediction with the first frame_to_box
    """
    index = str(get_trash_first_time(prediction))
    clean_frame_to_box = prediction['frame_to_box'][index]
    timed_prediction = {'time_index':int(index),'frame_to_box':clean_frame_to_box,'id':prediction['id'],'label':prediction['label']}    
    return timed_prediction

In [ ]:
# A single timed_prediction
timed_prediction = get_clean_timed_prediction(prediction)
timed_prediction

In [ ]:
# All timed predictions
for prediction in jsonPrediction['detected_trash']:
    print(prediction)
    print(get_timed_prediction(prediction))

In [ ]:
def get_df_prediction(json_prediction):
    timed_prediction_list = []
    for prediction in json_prediction['detected_trash']:
        timed_prediction_list.append(get_timed_prediction(prediction))
    df_prediction = pd.DataFrame(timed_prediction_list)
    return df_prediction

In [ ]:
df_predictions = get_df_prediction(jsonPrediction)
df_predictions.head()

In [ ]:
df_prediction.apply(lambda x: gps_points_filled[int(x.time_index)],axis=1)

In [ ]:
time_index = get_trash_time_index(prediction,media_fps)
trash_gps = gps_points_filled[time_index]
shape_trash_gps = long_lat_to_shape_point(trash_gps)
geo_2154 = transform_geo(shape_trash_gps)
geo_2154_trash_gps = {'Time': shape_trash_gps['Time'], 'the_geom': geo_2154,'Latitude':shape_trash_gps['Latitude'],'Longitude':shape_trash_gps['Longitude'], 'Elevation': shape_trash_gps['Elevation']}

In [ ]:
trash_gps = gps_points_filled[time_index]
shape_trash_gps = long_lat_to_shape_point(trash_gps)
geo_2154 = transform_geo(shape_trash_gps)
geo_2154_trash_gps = {'Time': shape_trash_gps['Time'], 'the_geom': geo_2154,'Latitude':shape_trash_gps['Latitude'],'Longitude':shape_trash_gps['Longitude'], 'Elevation': shape_trash_gps['Elevation']}

In [ ]:
def get_df_trash_gps(json_prediction:dict)->list:
    geo_2154_trash_gps_list = []
    for prediction in json_prediction['detected_trash']:
        time_index = get_trash_time_index(prediction,4)
        trash_gps = gps_points_filled[time_index]
        shape_trash_gps = long_lat_to_shape_point(trash_gps)
        geo_2154 = transform_geo(shape_trash_gps)
        geo_2154_trash_gps = {'Time': shape_trash_gps['Time'], 'the_geom': geo_2154,'Latitude':shape_trash_gps['Latitude'],'Longitude':shape_trash_gps['Longitude'], 'Elevation': shape_trash_gps['Elevation']}
        geo_2154_trash_gps_list.append(geo_2154_trash_gps)
        df_trash_gps = pd.DataFrame(geo_2154_trash_gps_list)
    return df_trash_gps

In [150]:
def get_df_trash_gps(df_predictions:pd.DataFrame,gps_points_filled:list)->pd.DataFrame:
    geo_2154_trash_gps_list = []
    time_indexes = df_predictions['time_index']
    for time_index in time_indexes:
        trash_gps = gps_points_filled[time_index]
        shape_trash_gps = long_lat_to_shape_point(trash_gps)
        geo_2154 = transform_geo(shape_trash_gps)
        geo_2154_trash_gps = {'Time': shape_trash_gps['Time'], 'the_geom': geo_2154,'Latitude':shape_trash_gps['Latitude'],'Longitude':shape_trash_gps['Longitude'], 'Elevation': shape_trash_gps['Elevation']}
        geo_2154_trash_gps_list.append(geo_2154_trash_gps)
        df_trash_gps = pd.DataFrame(geo_2154_trash_gps_list)
    return df_trash_gps

In [151]:
df_trash_gps = get_df_trash_gps(df_predictions)

In [152]:
df_trash_gps

,Time,the_geom,Latitude,Longitude,Elevation
0,2020-02-28 11:52:07+00:00,POINT (340987.7644429434 6289760.231710243),43.615853,-1.448500,53.203
1,2020-02-28 11:52:07+00:00,POINT (340987.7644429434 6289760.231710243),43.615853,-1.448500,53.203
2,2020-02-28 11:52:10+00:00,POINT (340989.4878405507 6289761.392383807),43.615865,-1.448480,51.617
3,2020-02-28 11:52:12+00:00,POINT (340990.7390486228 6289762.067635369),43.615871,-1.448465,51.140
4,2020-02-28 11:52:13+00:00,POINT (340991.5906244033 6289762.253372751),43.615873,-1.448454,51.082
5,2020-02-28 11:52:14+00:00,POINT (340992.3492124217 6289762.511143766),43.615876,-1.448445,51.401
6,2020-02-28 11:52:17+00:00,POINT (340994.2518469578 6289763.69510499),43.615888,-1.448423,50.899
7,2020-02-28 11:52:18+00:00,POINT (340994.6874808325 6289764.249378612),43.615893,-1.448418,50.447
8,2020-02-28 11:52:19+00:00,POINT (340995.5478903474 6289764.735170593),43.615898,-1.448407,50.217
9,2020-02-28 11:52:19+00:00,POINT (340995.5478903474 6289764.735170593),43.615898,-1.448407,50.217


In [ ]:
for time_index in df_predictions['time_index']:
    print(time_index)

In [ ]:
df_trash_gps = get_df_trash_gps(jsonPrediction)

In [ ]:
df_trash_gps.head()

In [ ]:
df_predictions.head()

In [154]:
def get_df_data(df_predictions:pd.DataFrame,df_trash_gps:pd.DataFrame)->pd.DataFrame:
    df_data = pd.concat([df_predictions,df_trash_gps],axis=1)
    return df_data

In [155]:
df_data = get_df_data(df_predictions,df_trash_gps)

In [156]:
df_data.head()

,time_index,frame_to_box,id,label,Time,the_geom,Latitude,Longitude,Elevation
0,0,"[0.27, 0.12, 0.3, 0.18]",0,fragments,2020-02-28 11:52:07+00:00,POINT (340987.7644429434 6289760.231710243),43.615853,-1.448500,53.203
1,0,"[0.28, 0.45, 0.29, 0.48]",1,fragments,2020-02-28 11:52:07+00:00,POINT (340987.7644429434 6289760.231710243),43.615853,-1.448500,53.203
2,3,"[0.18, 0.43, 0.2, 0.48]",2,fragments,2020-02-28 11:52:10+00:00,POINT (340989.4878405507 6289761.392383807),43.615865,-1.448480,51.617
3,5,"[0.15, 0.15, 0.18, 0.21]",3,fragments,2020-02-28 11:52:12+00:00,POINT (340990.7390486228 6289762.067635369),43.615871,-1.448465,51.140
4,6,"[0.32, 0.11, 0.35, 0.17]",4,fragments,2020-02-28 11:52:13+00:00,POINT (340991.5906244033 6289762.253372751),43.615873,-1.448454,51.082
